In [ ]:
import openai
import os
from pprint import pprint
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationEntityMemory
from langchain.memory import CombinedMemory

# read local .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 


# setup azure openai api
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OAI_ENDPOINT")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key  = os.getenv('AZURE_OAI_KEY')

llm_name = "gpt-3.5-turbo"
deployment_name_gpt = os.getenv("AZURE_OAI_MODEL_GPT_3")
deployment_name_ada = os.getenv("AZURE_OAI_MODEL_ADA")

os.environ["OPENAI_API_TYPE"] = openai.api_type
os.environ["OPENAI_API_BASE"] = openai.api_base
os.environ["OPENAI_API_KEY"] = openai.api_key
os.environ["OPENAI_API_VERSION"] = openai.api_version

In [ ]:
# The model
llm = AzureOpenAI(deployment_name=deployment_name_gpt)

In [ ]:

# Memory
conv_memory = ConversationBufferWindowMemory(
    k=3,
    input_key="input",
)

summary_memory = ConversationSummaryMemory(
    llm=llm,
    memory_key='summary',
    input_key="input",
)

# Combined
memory = CombinedMemory(memories=[conv_memory, summary_memory])
TEMPLATE = """
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation so far:
{summary}

Last 3 lines of conversation:
{history}

Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["summary", "history", "input"], template=TEMPLATE
)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=memory,
    prompt=PROMPT,
)

In [ ]:
pprint(conversation.run(input="Hello! How are you doing?"))